In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F 
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

torch.set_printoptions(linewidth=120)
torch.set_grad_enabled(True)

from torch.utils.tensorboard import SummaryWriter

In [2]:
def get_num_correct(preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

In [3]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1,out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
        self.fc1 = nn.Linear(12*4*4, 120)
        self.fc2 = nn.Linear(120, 60)
        self.out = nn.Linear(60, 10)

    def forward(self, t):
        t = F.relu(self.conv1(t))
        t = F.max_pool2d(t, 2, 2)
        
        t = F.relu(self.conv2(t))
        t = F.max_pool2d(t, 2, 2)

        t = t.flatten(start_dim=1)
        t = F.relu(self.fc1(t))

        t = F.relu(self.fc2(t))

        t = self.out(t)

        return t


In [4]:
train_set = torchvision.datasets.FashionMNIST(
    root='./data',
    train=True,
    download=False,
    transform=transforms.Compose([
        transforms.ToTensor()
    ])
)

train_loader = torch.utils.data.DataLoader(train_set, batch_size=100, shuffle=True)

In [6]:
tb = SummaryWriter()

network = Network()
images, labels = next(iter(train_loader))
grid = torchvision.utils.make_grid(images)

tb.add_image('images', grid)
tb.add_graph(network, images)
tb.close()

In [8]:
# The training Loop
network = Network()
train_loader = torch.utils.data.DataLoader(train_set, batch_size=100, shuffle=True)
optimizer = optim.Adam(network.parameters(), lr=0.01)

for epoch in range(1):

    total_loss = 0
    total_correct = 0

    for batch in train_loader:
        images, labels = batch
        
        preds = network(images)
        loss = F.cross_entropy(preds, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_correct += get_num_correct(preds, labels)

    print("epoch", epoch, "total_correct:", total_correct, "loss", total_loss)

epoch 0 total_correct: 47590 loss 328.35882860422134


In [10]:
# version 2.0

network = Network()
train_loader = torch.utils.data.DataLoader(train_set, batch_size=100, shuffle=True)
optimizer = optim.Adam(network.parameters(), lr=0.01)
# 
images, labels = next(iter(train_loader))
grid = torchvision.utils.make_grid(images)

tb = SummaryWriter()
tb.flush()
tb.add_image('images', grid)
tb.add_graph(network, images)
#  


for epoch in range(10):

    total_loss = 0
    total_correct = 0

    for batch in train_loader:
        images, labels = batch
        
        preds = network(images)
        loss = F.cross_entropy(preds, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_correct += get_num_correct(preds, labels)


# 
    # tb.add_scalars('train curve',{'Loss':total_loss,'Number Correct':total_correct,"Accuracy": total_correct/len(train_set)},global_step=epoch)
    tb.add_scalar('Loss', total_loss, epoch)
    tb.add_scalar('Number Correct', total_correct, epoch)
    tb.add_scalar('Accuracy', total_correct/len(train_loader), epoch)
    
    tb.add_histogram('conv1.bias', network.conv1.bias, epoch)
    tb.add_histogram('conv1.weight', network.conv1.weight, epoch)
    tb.add_histogram('conv1.weight.grad', network.conv1.weight.grad, epoch)
# 
    print("epoch", epoch, "total_correct:", total_correct, "loss", total_loss)

epoch 0 total_correct: 46962 loss 340.21916511654854
epoch 1 total_correct: 51271 loss 235.1044681444764
epoch 2 total_correct: 52147 loss 214.08032236993313
epoch 3 total_correct: 52331 loss 205.61190839111805
epoch 4 total_correct: 52540 loss 200.6478017270565
epoch 5 total_correct: 52785 loss 195.12828519940376
epoch 6 total_correct: 53013 loss 189.12889145314693
epoch 7 total_correct: 53066 loss 188.5572074651718
epoch 8 total_correct: 53112 loss 184.02178723365068
epoch 9 total_correct: 53384 loss 180.03217387199402
